<div style="width:100%; background-color: #000041"><a target="_blank\" href="http://university.yugabyte.com\"><img src="assets/YBU_Logo.webp" /></a></div><br>

> **YugabyteDB YCQL Development**
>
> Enroll for free at [Yugabyte University](https://university.yugabyte.com/courses/yugabytedb-ycql-development).
>

# Query-driven data model: Secondary indexes
In this notebook, you will learn how to create secondary indexes to not only improve query performance, but also remove unnecessary tables from the data model.

### Import the notebook variables 

> Requirements:
>
> You must first create the variables in the `01_Setup.ipynb` notebook.
>

The following Python cell reads the stored variables created in the `01_Setup.ipynb` notebook. 

- To run the script, select Execute Cell (Play Arrow) in the left gutter of the cell.

In [ ]:
%store -r MY_DB_NAME
%store -r MY_YB_PATH
%store -r MY_YB_PATH_DATA
%store -r MY_GITPOD_WORKSPACE_URL
%store -r MY_HOST_IPv4_01
%store -r MY_HOST_IPv4_02
%store -r MY_HOST_IPv4_03
%store -r MY_NOTEBOOK_DIR
%store -r MY_TSERVER_WEBSERVER_PORT
%store -r MY_NOTEBOOK_DATA_FOLDER
%store -r MY_YB_MASTER_HOST_GITPOD_URL
%store -r MY_YB_TSERVER_HOST_GITPOD_URL
%store -r MY_DATA_DDL_FILE
%store -r MY_DATA_DML_FILE

#### Operators

JSONB is considered the best way to utilize complex data structures since in YCQL, JSONB is searchable. This is not true of the other collections in YCQL. Also note that collections can be used in JSON as well. Run the following cell and note the pattern necessary to write a JSON object to a table in YCQL.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  select category, product_name, product_id, price, 
    (sku_details->'tags'->>0) as tag1, 
    (sku_details->'tags'->>1) as tag2,
    (sku_details->'tags'->>2) as tag3,
    (sku_details->'tags'->>3) as tag4,
    (sku_details->'tags'->>4) as tag5,
    (sku_details->'tags'->>5) as tag6,
    (sku_details->'colors'->>0) as color1, 
    (sku_details->'colors'->>1) as color2,
    (sku_details->'colors'->>2) as color3,
    (sku_details->'colors'->>3) as color4,
    (sku_details->'colors'->>4) as color5,
    (sku_details->'colors'->>5) as color6,
    (sku_details->'colors'->>6) as color7
  from tbl_products_by_category
  where (sku_details->>'kid_friendly')='true' 
  ;
"

### insert JSONB
benefits

```

```

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  insert into tbl_products_by_category (category, product_name, product_id, brand, price, discount, description, gtin, sku_details) values ('H20','Talc 10',62373,'Yeah',19.99,7,'2 liter ','006237337326','{\"sku\":\"YH_62373\",\"country\":\"UK\",\"tags\":[\"water\",\"bottle\",\"everyday\",\"workout\"],\"colors\":[\"blue\",\"green\",\"orange\",\"red\"],\"dimensions\":{\"dm_unit\":\"in\",\"dm_length\":\"19\",\"dm_width\":\"19\",\"dm_height\":\"19\"}, \"kid_friendly\":true}');
 "

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  select category, product_name, product_id, brand, sku_details
  from tbl_products_by_category
   where (sku_details->>'sku')='YH_62373' 
  ;
"

> **Question**
>
> What is the order of the keys in the `sku_details` column?
>
> **Answer**
>
> The keys are lexicographically ordered.

### add a key-value pair to JSONB

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
 update tbl_products_by_category
   set sku_details->'rating' = '3'
   where category = 'H20'
     and product_name = 'Talc 10'
     and product_id = 62373 
    ;
 "

Confirm the new key-value pair.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  select category, product_name, product_id, brand, sku_details
  from tbl_products_by_category
   where (sku_details->>'sku')='YH_62373' 
  ;
"

### update value of key-value pair

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
 update tbl_products_by_category
   set sku_details->'rating' = '5'
   where category = 'H20'
     and product_name = 'Talc 10'
     and product_id = 62373 
    ;
 "

Confirm the update.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  select category, product_name, product_id, brand, sku_details
  from tbl_products_by_category
   where (sku_details->>'sku')='YH_62373' 
  ;
"

### update JSONB

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "

  update tbl_products_by_category
   set sku_details = '{\"sku\":\"YH_62373\",\"country\":\"UK\",\"tags\":[\"water\",\"bottle\",\"everyday\",\"workout\"],\"colors\":[\"purple\",\"green\",\"orange\",\"red\"],\"dimensions\":{\"dm_unit\":\"in\",\"dm_length\":\"18\",\"dm_width\":\"18\",\"dm_height\":\"18\"}, \"kid_friendly\":false}'
   where category = 'H20'
     and product_name = 'Talc 10'
     and product_id = 62373  
  ;
 "

Confirm the update. 

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  select category, product_name, product_id, brand, sku_details
  from tbl_products_by_category
   where (sku_details->>'sku')='YH_62373' 
  ;
"

### query plan

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  explain select category, product_name, product_id, brand, description, price, gtin, sku_details->> 'sku' as sku
  from tbl_products_by_category
  where
    sku_details->>'sku' = 'YH_62373' 
  ;
"

#### JSONB Index

TODO: 

- key-value can  partition key and clustering keys, but will function as index key and to use index, must include full key in predicate.
- cannot be an index key in whole
- cannot be part of include
- cannot be part of where expression

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  drop index if exists idx_products_by_catgegory_jsonb
  ;
"

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  create index if not exists idx_products_by_catgegory_jsonb
  on tbl_products_by_category ( (
     sku_details->>'sku'
      ) )
  include (
    brand,
    price,
    description,
    gtin
    )
  ;
"

Query plan

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_DB_NAME"   # Query plan: Sequential scan  
YB_PATH=${1}
DB_NAME=${2}  
cd $YB_PATH/bin

# DB_NAME=ks_ybu
./ycqlsh -r -k $DB_NAME -e "
  explain select category, product_name, product_id, brand, description, price, gtin, sku_details->> 'sku' as sku
  from tbl_products_by_category
  where (sku_details->>'sku')='YH_62373' 
  ;
"

#### Select a YB-TServer host
Set the host variable for one of the nodes. All three nodes in the cluster are running a Tablet Server (YB-TServer). You can comment/uncomment lines 7-9 as needed.

In [ ]:
%%bash -s "$MY_HOST_IPv4_01" "$MY_HOST_IPv4_02" "$MY_HOST_IPv4_03" --out MY_HOST_IPv4
HOST_IPv4_01=$( echo "${1}" | tr -d " ")
HOST_IPv4_02=$( echo "${2}" | tr -d " ")
HOST_IPv4_03=$( echo "${3}" | tr -d " ")

# change the hosts for different tablet leaders
MY_HOST_IPv4=$HOST_IPv4_01
#MY_HOST_IPv4=$HOST_IPv4_02
#MY_HOST_IPv4=$HOST_IPv4_03

echo ${MY_HOST_IPv4}

Store the select host variable.

In [ ]:
%store MY_HOST_IPv4
print(MY_HOST_IPv4)

Save the index name as a variable.

In [ ]:
MY_OBJECT_NAME="idx_products_by_catgegory_jsonb"
%store MY_OBJECT_NAME
print(MY_OBJECT_NAME)

Grep the index_id for the index using `curl` and `jq`.

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4"  "$MY_DB_NAME"  "$MY_TSERVER_WEBSERVER_PORT"  --out MY_INDEX_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
DB_NAME=$( echo "${3}" | tr -d " ")
TSERVER_WEBSERVER_PORT=$( echo "${4}" | tr -d " ")


MY_URL="http://${HOST_IPv4}:${TSERVER_WEBSERVER_PORT}/metrics"

MY_INDEX_ID=`curl -s --compressed ${MY_URL} | jq -r 'limit(1;  .[] | select(.attributes.namespace_name=="'${DB_NAME}'" and .type=="tablet" and .attributes.table_name=="'${OBJECT_NAME}'") |  .attributes.table_id) '`

echo ${MY_INDEX_ID}

Store the index_id for the index.

In [ ]:
%store MY_INDEX_ID
print(MY_INDEX_ID)

Get the tablet_id for the tablet leader for the select node host.

In [ ]:
%%bash -s "$MY_OBJECT_NAME" "$MY_HOST_IPv4" --out MY_INDEX_TABLET_ID
OBJECT_NAME=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")

MY_URL="http://${HOST_IPv4}:8200/metrics"

MY_INDEX_TABLET_ID=`curl -s --compressed ${MY_URL} | jq --raw-output ' .[] | select(.attributes.namespace_name=="ks_ybu" and .type=="tablet" and .attributes.table_name=="'$OBJECT_NAME'") | {tablet_id: .id, metrics: .metrics[] | select(.name == ("is_raft_leader") ) | select(.value == 1) } | select(.tablet_id) | {tablet_id} | .tablet_id '`

echo ${MY_INDEX_TABLET_ID}

Store the tablet_id for the tablet leader.

In [ ]:
%store MY_INDEX_TABLET_ID
print(MY_INDEX_TABLET_ID)

Flush the WAL file to a SST file for the given index_id.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_HOST_IPv4" "$MY_INDEX_ID"  # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
HOST_IPv4=$( echo "${2}" | tr -d " ")
INDEX_ID=$( echo "${3}" | tr -d " ")
cd $YB_PATH/bin

./yb-admin -init_master_addrs ${HOST_IPv4}:7100 flush_table_by_id ${INDEX_ID} 600

Dump and decode the SST file in human-readable form.

> Note:
>
> If the following does **NOT** dump the SST file, it is most likely that there are not any rows written to this tablet. To resolve this issue, you need to select a different Tablet Server host. Return back to **Select a YB-TServer host** and select a different node host.

In [ ]:
%%bash -s "$MY_YB_PATH" "$MY_YB_PATH_DATA" "$MY_INDEX_ID" "$MY_INDEX_TABLET_ID" # Import file path of Yugabyte and DB name
YB_PATH=$( echo "${1}" | tr -d " ")
YB_PATH_DATA=$( echo "${2}" | tr -d " ")
INDEX_ID=$( echo "${3}" | tr -d " ")
INDEX_TABLET_ID=$( echo "${4}" | tr -d " ")

cd $YB_PATH/bin/

INDEX_ID_PATH=${YB_PATH_DATA}/node-1/disk-1/yb-data/tserver/data/rocksdb/table-${INDEX_ID}/tablet-${INDEX_TABLET_ID}

# ls -l  ${TABLE_ID_PATH}

./sst_dump --command=scan --file=${INDEX_ID_PATH} --output_format=decoded_regulardb 

In the preceding cell, note the syntax that is required to create an index using a JSON key. Also note the syntax used to search by the key in a JSON object.

If the row is still visible, wait a few more seconds to run the preceding cell. This will verify that the row has expired as expected.

---
# All done!
In this lab, you completed the following:

- Setup
  - Created the `ks_ybu` database with `ycqlsh`
  - Created tables and loaded data using DDL and DML scripts